Este es el código que se utilizó para hacer el ETL Job, a pesar de que no fue posible utilizarlo por un problema con los permisos del rol IAM LabRole por lo que se decidió hacerlo localmente con el código de Python usando PySpark que se encuenta en Proccess_SIATA.ipynb

In [ ]:
import sys
from awsglue.transforms import *
from awsglue.utils import getResolvedOptions
from pyspark.context import SparkContext
from awsglue.context import GlueContext
from awsglue.job import Job

## @params: [JOB_NAME]
args = getResolvedOptions(sys.argv, ['JOB_NAME'])

sc = SparkContext()
glueContext = GlueContext(sc)
spark = glueContext.spark_session
job = Job(glueContext)
job.init(args['JOB_NAME'], args)





input_path = "s3://tu-bucket/siata/raw/"
output_path = "s3://tu-bucket/siata/trusted/"


json_files = ["co", "no", "so2", "no2", "ozono", "pm1", "pm10", "pm25"]

for file_name in json_files:

    df = spark.read.json(f"{input_path}{file_name}/")


    df = df.withColumn("datos", explode(df["datos"]))


    df = df.select(
        "latitud",
        "codigoSerial",
        "datos.variableConsulta",
        "datos.fecha",
        "datos.calidad",
        "datos.valor",
        "nombre",
        "nombreCorto",
        "longitud"
    )


    df.write.mode("overwrite").parquet(f"{output_path}{file_name}/")
        
job.commit()